In [22]:
type Parser a = String -> [(a,String)]

In [23]:
satisfy :: (Char -> Bool) -> Parser Char
satisfy p (c:cs) | p c = [(c,cs)] 
satisfy _  _           = []

In [24]:
anyChar = satisfy (const True) -- statisfy (\_ -> True)
char x = satisfy (x==)
oneOf xs = satisfy (`elem` xs)

In [25]:
oneOf ['a','b','c'] "abasdf"
oneOf ['a','b','c'] "basdf"
oneOf ['a','b','c'] "rrasdf"

[('a',"basdf")]

[('b',"asdf")]

[]

In [26]:
import Data.Char
space = satisfy isSpace
alphaNum = satisfy isAlphaNum
letter = satisfy isAlpha
lower = satisfy isLower
upper = satisfy isUpper
digit = satisfy isDigit

아래의 실행예처럼 주어진 문자열과 정확히 일치하는 만큼을 분석하여 처리하는 `string` 함수를 작성하라. (다음 과제의 첫 문제가 될 내용이다)
```haskell
string "abc" "abcdef" == [("abc","def")]
```

In [27]:
string :: String -> Parser String 
string ""     cs            = [("",cs)]
string (x:xs) (c:cs) | x==c = [(x:xs',cs') | (xs',cs')<-string xs cs]
string _      _             = []

In [28]:
string "abc" "abcdef"

[("abc","def")]

In [29]:
string "abc" "cccdef"

[]

----

문법분석 함수 many1과 many를 가장 길게 (더 정확히는 가장 여러 번 반복되는 호수로) 매칭되는 경우만 결과로

In [30]:
-- parse one or more times
many1 :: Parser a -> Parser [a]
many1 p cs = take 1 $
             [(x:xs, cs'') | (x,cs')<-ps, (xs,cs'')<-many1 p cs']
             ++
             [([x], cs') | (x,cs')<-ps] -- 한번 처리한 결과 x를 리스트로
           where ps = p cs

-- parse zero or more time
many :: Parser a -> Parser [a]
many p cs = take 1 $ many1 p cs ++ [([],cs)] 

In [31]:
spaces = many space

In [32]:
space  "    abcd"
spaces "    abcd"

[(' ',"   abcd")]

[("    ","abcd")]

## 십진수 양의 정수를 분석하기

In [33]:
-- 맨 앞에는 0이 아닌 숫자, 그 다음에는 아무 십진수 숫자나

firstDigit = oneOf ['1'..'9'] -- 맨 앞글자를 처리 (딱 1개 글자)
digits = many digit       -- 뒷글자들을 처리 (0개 이상 글자)

In [34]:
(>>=) :: Parser a -> (a -> Parser b) -> Parser b
p >>= f = \cs -> [(y,cs'') | (x, cs')<-p cs, (y,cs'') <- f x cs']

infixr 1 >>=

In [35]:
return :: b -> Parser b
return v = \cs -> [(v,cs)] 

In [46]:
posint :: Parser Int
posint = firstDigit >>= \x  ->
         digits     >>= \xs ->
         return (read(x:xs))

In [47]:
posint "123 asdf"

[(123," asdf")]

# 자연수 Parser와  정수 Parser

이제 자연수 Parser와 정수 Parser를 만들어 보자. 자연수는 양의 정수이거나 0이며 정수는
자연수이거나 음의 정수이므로 이들 각각에 해당하는 Parser를 만들어 결과가 그 중 어느 것이든 될 수 있는 Parser를 만들면 된다. 이를 위해 필요한 Parser에 대한 연산자를 정의하자.

In [48]:
(<|>) :: Parser a -> Parser a -> Parser a
p1 <|> p2 = \cs -> take 1 $ p1 cs ++ p2 cs

infixl 3 <|>

In [49]:
nat :: Parser Int
nat = zero <|> posint

int :: Parser Int
int = nat <|> negint

zero :: Parser Int
zero   = char '0' >>= \_ ->
         return 0

negint :: Parser Int
negint = char '-' >>= \_ ->
         posint   >>= \n ->
         return (-n)

----

In [52]:
many1 p = p >>= \x ->
          (many1 p >>= \xs -> return (x:xs)) <|> return [x]

many p = many1 p <|> return []

In [53]:
token :: Parser a -> Parser a
token p = spaces >>= \_ ->
          p      >>= \v ->
          spaces >>= \_ ->
          return v

In [54]:
integer = token int
lowercase = token lower

In [55]:
int "-2"
lower "a"

[(-2,"")]

[('a',"")]

In [56]:
integer " 123 "
integer " -123 "
integer " 0 "

[(123,"")]

[(-123,"")]

[(0,"")]

In [57]:
lowercase "   b  "
lowercase "   A  "

[('b',"")]

[]

In [58]:
tok = token . char
tokDigit = token digit

----
소수점이 들어간 수의 단어분석

In [67]:
fromIntParser p = p >>= \n -> return(fromIntegral n)
{-
negint' = negint >>= \n -> return(fromIntegral n)
int'    = int    >>= \n -> return(fromIntegral n)
zero'   = zero   >>= \n -> return(fromIntegral n)
-}
negint' = fromIntParser negint
int'    = fromIntParser int
zero'   = fromIntParser zero

float, posfloat, negfloat :: Parser Float

negfloat = char '-' >>= \_ ->
           posfloat >>= \v ->
           return (-v)

posfloat = (int' <|> zero') >>= \v ->
           char '.'         >>= \_  ->
           many1 digit      >>= \cs ->
           return (v + read("0."++cs))

float = negfloat <|> negint' <|> posfloat <|> int' <|> zero'

floating = token float

In [73]:
floating "  0.15"
floating "  12.25"
floating "  12. 25"
floating "  -0.25"
floating "  0. 25"
floating "  -0. 25"
floating "  -20.34"
floating "  -20. 34"

[(0.15,"")]

[(12.25,"")]

[(12.0,". 25")]

[(-0.25,"")]

[(0.0,". 25")]

[]

[(-20.34,"")]

[(-20.0,". 34")]

----

애매한 문법
```
expr ::= expr '+' expr | expr '*' expr | digit | '(' expr ')'
digit ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

왼쪽재귀를 없앤 문법 (하지만 연산자 우선순위는 지정 안되어 있음)
```
expr ::= digit '+' expr | digit '*' expr | digit | '(' expr ')'
digit ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

연산자 우선순위까지 지정하는 문법 (하지만 결합 방향은 우결합만 가능)
```
term   ::= digit   |  '(' expr ')'
factor ::= term    |  term   '*' factor
expr   ::= factor  |  factor '+' expr
```

연산자 결합 방향을 문법분석 후 처리하기 용이하도록 반복되는 부분을 Kleene 연산자로 묶어줌
```
term   ::= digit  |  '(' expr ')'
factor ::= term   ( '*' term   )*
expr   ::= factor ( '+' factor )*
```

In [122]:
-- 나무구조의 추상문법(abstract syntax)에는 연산자 우선순위나 괄호 등이 필요없다
-- 이미 나무구조로 어떤 식들이 어떻게 묶여 있는지 표현되기 때문
data Expr = Plus Expr Expr | Mult Expr Expr | Digit Char deriving Show

In [123]:
-- 재귀 하강 문법분석기 (recursive decendent parser)
digitExpr, term, factor, expr :: Parser Expr

digitExpr = tokDigit >>= \c ->
            return $ Digit c

-- term   ::= digit  |  '(' expr ')'
term   = digitExpr <|> ( tok '(' >>= \_ ->
                         expr    >>= \e ->
                         tok ')' >>= \_ ->
                         return e )

-- factor ::= term   ( '*' term   )*
factor = term                       >>= \e0 ->
         many ( tok '*' >>= \_ ->
                term    >>= \e ->
                return e          ) >>= \es ->
         return $ foldl Mult e0 es

-- expr   ::= factor ( '+' factor )*
expr   = factor                     >>= \e0 ->
         many ( tok '+' >>= \_ ->
                factor  >>= \e ->
                return e          ) >>= \es ->
         return $ foldl Plus e0 es

In [124]:
term "1"
term "(1+2)"

factor "1"
factor "1*2*3"

expr "1"
expr "1+2+3"
expr "1+2*3"
expr "1*2+3"

[(Digit '1',"")]

[(Plus (Digit '1') (Digit '2'),"")]

[(Digit '1',"")]

[(Mult (Mult (Digit '1') (Digit '2')) (Digit '3'),"")]

[(Digit '1',"")]

[(Plus (Plus (Digit '1') (Digit '2')) (Digit '3'),"")]

[(Plus (Digit '1') (Mult (Digit '2') (Digit '3')),"")]

[(Plus (Mult (Digit '1') (Digit '2')) (Digit '3'),"")]

# hw5 (12/5 밤까지, 6점)

위의 2칙연산 문법분석기를 4칙연산을 지원하도록 수정해 보라.